In [1]:
from urllib.request import urlopen, urlretrieve
from os.path import exists, join, isdir 
from os import makedirs
import sys
import pandas as pd
from astropy import units as u
from astropy.coordinates import SkyCoord
import numpy as np
from tqdm import tqdm
sys.path.append('../archive_digger/')
import archive_digger as ad

In [2]:
df = ad.get_harps_database(clobber=False)

Loaded: ../data/harps_db.csv


In [3]:
tois = ad.get_tois(clobber=False)
tois.shape

Loaded: ../data/TOIs.csv


(6160, 62)

In [ ]:
outdir_base = '../all_tois/v02/'
tois_unique = tois.drop_duplicates(subset='TIC ID')
ndata,rv_urls,comments = [],[],[]
for i,row in tqdm(tois_unique[::-1].iterrows()):
    # print(row)
    ra, dec = row['RA'], row['Dec']
    p1,p2 = str(row.TOI).split('.')
    toiid = p1.zfill(4)
    name = 'TOI'+toiid
    target_coord = SkyCoord(ra, dec, unit=(u.hourangle, u.deg))
    res = ad.query_target(target_coord, df, dist=10*u.arcsec, verbose=False)
    if isinstance(res, pd.DataFrame):
        r = res.iloc[0]
        nearest_obj = r['Target']
        print(nearest_obj)
        outdir = join(outdir_base, f'{name}_{nearest_obj}')
        if not isdir(outdir):
            makedirs(outdir)
        
        df_rv, url = ad.get_rv(r, col='Data products (csv)', return_url=True)
        fp_rv = join(outdir, f'{name}_{nearest_obj}.csv')
        if not exists(fp_rv):
            df_rv.to_csv(fp_rv, index=False)
            print("Saved: ", fp_rv)
        comments.append(row['Comments'])
        ndata.append(len(df_rv))
        rv_urls.append(url)
        fp_png = join(outdir, f'{name}_{nearest_obj}_fov.png')
        if not exists(fp_png):
            fig = ad.plot_fov(target_coord,res,fov_rad=60*u.arcsec,ang_dist=15*u.arcsec,
                     survey='DSS',verbose=True,outdir=outdir)
            fig.savefig(fp_png, bbox_inches="tight")
            print("Saved: ", fp_rv)

## results

In [20]:
from glob import glob

fl = glob('../all_tois/v02/*')

toiids, names, exofops, harps = [],[],[],[]
for f in sorted(fl, reverse=True):
    toi,name = f.split('/')[-1].split('_')
    base = 'https://www2.mpia-hd.mpg.de/homes/trifonov/GLS/gls.html?fend=1&url='
    harps_url = base+f'../Ver_02/{name}_RVs/{name}_HARPS_RVBank_ver02_rvsonly.dat&run'
    exofop_url = f'https://exofop.ipac.caltech.edu/tess/target.php?id=TOI-{toi[3:]}'
    toiids.append(toi)
    names.append(name)
    exofops.append(exofop_url)
    harps.append(harps_url)
    # print(toi,exofop_url,harps_url)

In [21]:
df_url = pd.DataFrame(np.c_[toiids,names,exofops,harps,rv_urls,ndata,comments],columns=['toi','name','exofop_url','harps_url','rv_url','ndata','comment'])
df_url

,toi,name,exofop_url,harps_url,rv_url,ndata,comment
0,TOI6978,GJ1247,https://exofop.ipac.caltech.edu/tess/target.ph...,https://www2.mpia-hd.mpg.de/homes/trifonov/GLS...,https://www2.mpia-hd.mpg.de/homes/trifonov/Ver...,33,low SNR
1,TOI6965,HD50554,https://exofop.ipac.caltech.edu/tess/target.ph...,https://www2.mpia-hd.mpg.de/homes/trifonov/GLS...,https://www2.mpia-hd.mpg.de/homes/trifonov/Ver...,1,interior to HD 50554 b; period may be double (...
2,TOI6835,K2-141,https://exofop.ipac.caltech.edu/tess/target.ph...,https://www2.mpia-hd.mpg.de/homes/trifonov/GLS...,https://www2.mpia-hd.mpg.de/homes/trifonov/Ver...,28,K2-141 b
3,TOI6833,K2-138,https://exofop.ipac.caltech.edu/tess/target.ph...,https://www2.mpia-hd.mpg.de/homes/trifonov/GLS...,https://www2.mpia-hd.mpg.de/homes/trifonov/Ver...,214,K2-138 e; marginal QLP detection in combined T...
4,TOI6806,TYC8459-487-1,https://exofop.ipac.caltech.edu/tess/target.ph...,https://www2.mpia-hd.mpg.de/homes/trifonov/GLS...,https://www2.mpia-hd.mpg.de/homes/trifonov/Ver...,6,period may be half; found in faint-star QLP se...
...,...,...,...,...,...,...,...
271,TOI0119,HIP31609,https://exofop.ipac.caltech.edu/tess/target.ph...,https://www2.mpia-hd.mpg.de/homes/trifonov/GLS...,https://www2.mpia-hd.mpg.de/homes/trifonov/Ver...,7,Potential multi; slight depth aperture correla...
272,TOI0118,HD219666,https://exofop.ipac.caltech.edu/tess/target.ph...,https://www2.mpia-hd.mpg.de/homes/trifonov/GLS...,https://www2.mpia-hd.mpg.de/homes/trifonov/Ver...,24,HD 219666 b
273,TOI0116,WASP-91,https://exofop.ipac.caltech.edu/tess/target.ph...,https://www2.mpia-hd.mpg.de/homes/trifonov/GLS...,https://www2.mpia-hd.mpg.de/homes/trifonov/Ver...,20,WASP-91 b
274,TOI0102,WASP-62,https://exofop.ipac.caltech.edu/tess/target.ph...,https://www2.mpia-hd.mpg.de/homes/trifonov/GLS...,https://www2.mpia-hd.mpg.de/homes/trifonov/Ver...,39,WASP 62 b


In [19]:
df_url.to_csv('../data/toi_with_harps_v2.csv',index=False)

In [26]:
df_url.iloc[5:20][['toi','name','comment','harps_url']].values

array([['TOI6726', 'HD216530', 'HIP 113103 c',
        'https://www2.mpia-hd.mpg.de/homes/trifonov/GLS/gls.html?fend=1&url=../Ver_02/HD216530_RVs/HD216530_HARPS_RVBank_ver02_rvsonly.dat&run'],
       ['TOI6713', 'HD104067',
        'interior to the known RV planet HD 104067 b; low SNR',
        'https://www2.mpia-hd.mpg.de/homes/trifonov/GLS/gls.html?fend=1&url=../Ver_02/HD104067_RVs/HD104067_HARPS_RVBank_ver02_rvsonly.dat&run'],
       ['TOI6709', 'HIP54597', 'low SNR',
        'https://www2.mpia-hd.mpg.de/homes/trifonov/GLS/gls.html?fend=1&url=../Ver_02/HIP54597_RVs/HIP54597_HARPS_RVBank_ver02_rvsonly.dat&run'],
       ['TOI6697', 'HD114082', 'single transit of HD 114082 b',
        'https://www2.mpia-hd.mpg.de/homes/trifonov/GLS/gls.html?fend=1&url=../Ver_02/HD114082_RVs/HD114082_HARPS_RVBank_ver02_rvsonly.dat&run'],
       ['TOI6695', 'TYC6570-1482-1',
        'single transit; possible additional transit at 1525.64811 TBJD; CTOI from Rafael Brahm',
        'https://www2.mpia-hd.mpg